## 라이브러리

In [18]:
# beautifulSoup4: 웹 페이지의 정보를 쉽게 스크랩할 수 있도록 기능을 제공하는 라이브러리
from bs4 import BeautifulSoup
# selenium: beautifulSoup4과 같지만 자바스크립트가 동적으로 만든 데이터를 크롤링하기 위함
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
# requests: HTTP 요청을 보낼 수 있도록 기능을 제공하는 라이브러리
import requests
# re: 정규 표현식을 사용할때, 사용하는 파이썬 내장 모듈
import re
import pandas as pd
import numpy as np
import os
import csv
from time import sleep

import warnings 
warnings.filterwarnings('ignore')

## 셀레니움으로 별점 및 리뷰 저장

In [19]:
# webdriver가 위치한 경로로 설정
driver = webdriver.Chrome('/Users/boyeonjang/git/kyobobook-review/chromedriver')

In [ ]:
numbers = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'Q', 'R', 'S', 'T', 'U', 'V', 'a', 'b', 'c', 'd', 'e', 'f', 'g', '08', 'h', 'i']

In [ ]:
#(추가) 분야별로 for문
driver.get("http://www.kyobobook.co.kr/bestSellerNew/bestseller.laf?mallGb=KOR&linkClass=c&range=1&kind=0&orderClick=DAb")

for i in range(1,21): # top 20
  driver.find_element(By.XPATH, f'''//*[@id="main_contents"]/ul/li[{i}]/div[2]/div[2]/a/strong''').click() # 책 제목 클릭

  sleep(3)

  driver.find_element(By.XPATH, '''//*[@id="event_info"]/li[3]/a''').click() # 리뷰 클릭

  sleep(3)

  for idx in range(1,6): # 리뷰 5개씩

    # one_review = driver.find_element(By.XPATH, f'''//*[@id="box_detail_review"]/ul/li[{idx}]/div[1]''')

    rating_xpath = driver.find_element(By.XPATH, f'''//*[@id="box_detail_review"]/ul/li[{idx}]/div[1]/dl/dd[3]/span''')
    text_xpath = driver.find_element(By.XPATH, f'''//*[@id="box_detail_review"]/ul/li[{idx}]/div[1]/dl/dd[5]/div''')
    date_xpath = driver.find_element(By.XPATH, f'''//*[@id="box_detail_review"]/ul/li[{idx}]/div[1]/dl/dd[1]''')
    
    #(추가)csv로 저장
    rating = rating_xpath.text
    text = text_xpath.text
    date = date_xpath.text
    print("rating: ", rating)
    print("text: ", text)
    print("date: ", date)
    sleep(5)

    #(추가) 다음페이지 넘기기

  driver.close()

  driver.back()
  sleep(2)

In [ ]:
driver.get('http://www.kyobobook.co.kr/bestSellerNew/bestseller.laf?mallGb=KOR&linkClass=B&range=1&kind=2&orderClick=DAd')

for book in range(1, 21): # top 20
  driver.find_element(By.XPATH, f'''//*[@id="main_contents"]/ul/li[{book}]/div[2]/div[2]/a/strong''').click() # 책 제목 클릭
  driver.implicitly_wait(time_to_wait=10) # 파싱이 되면 바로 다음 코드로 넘어간다 파싱이 안될 경우 최대 10초를 기다린다
  
  title = driver.find_element(By.XPATH, '''//*[@id="container"]/div[2]/form/div[1]/h1/strong''')
  print('title: ', title)

  driver.find_element(By.XPATH, '''//*[@id="event_info"]/li[3]/a''').click() # 리뷰 클릭
  driver.implicitly_wait(time_to_wait=10)

  # 페이지가 다음으로 넘어가면
  for page in range(1, 13):
    for idx in range(1, 6): # 리뷰 5개씩
      rating_xpath = driver.find_element(By.XPATH, f'''//*[@id="box_detail_review"]/ul/li[{idx}]/div[1]/dl/dd[3]/span''') # 별점
      text_xpath = driver.find_element(By.XPATH, f'''//*[@id="box_detail_review"]/ul/li[{idx}]/div[1]/dl/dd[5]/div''') # 리뷰 글
      date_xpath = driver.find_element(By.XPATH, f'''//*[@id="box_detail_review"]/ul/li[{idx}]/div[1]/dl/dd[1]''') # 날짜
      
      #(추가)csv로 저장
      rating = rating_xpath.text
      text = text_xpath.text
      date = date_xpath.text
      print('rating: ', rating)
      print('text: ', text)
      print("date: ", date)
      driver.implicitly_wait(time_to_wait=10)
      
      # 3개 열을 가진 pandas DataFrame으로 바꿔준 뒤 엑셀 파일로 저장
      df = pd.DataFrame({'title':title, 'rating':rating, 'text':text}, index=0)
      df.to_csv("best_books.csv", encoding='utf-8-sig')

    # 다음페이지 넘기기
    driver.find_element(By.XPATH, f'''//*[@id="box_detail_review"]/div[3]/div/a[{page}]''').click()
  
  driver.close()
  driver.back()